In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

from  vectorizing_funcs import *
df = pd.read_csv('../all_data.csv', sep = '|', error_bad_lines=False, index_col=False, dtype='unicode')
df.head()

,SubjectID,form_name,feature_name,feature_value,feature_unit,feature_delta
0,533,Demographic,Gender,F,NaN,0.0
1,533,Demographic,Age,65,NaN,0.0
2,533,Demographic,Race,White,NaN,0.0
3,533,ALSHX,onset_delta,-1023,NaN,0.0
4,533,ALSHX,diag_delta,-44,NaN,0.0


In [2]:
feature_names = df[["form_name", "feature_name"]].drop_duplicates()
feature_names.to_csv('../feature_names.csv', sep='|', index=False)

In [3]:
feature_values = df[["form_name", "feature_name", "feature_value"]].drop_duplicates()
feature_values = feature_values[np.isnan(feature_values.feature_value.convert_objects(convert_numeric=True))]
feature_values.to_csv('../feature_values.csv', sep='|', index=False)

In [5]:
feature_values = df[df.form_name == 'Lab Test']
feature_values = feature_values[~np.isnan(feature_values.feature_value.convert_objects(convert_numeric=True))]
by_subject = feature_values.groupby(["feature_name", "SubjectID"])
features_with_multiple_visits = by_subject.filter(lambda x: len(x)>2)
by_subject = features_with_multiple_visits.groupby("feature_name").SubjectID.nunique()
by_subject.sort(ascending=False)
by_subject[:30]


feature_name
Creatinine                   2391
Blood Urea Nitrogen (BUN)    2390
ALT(SGPT)                    2390
Sodium                       2390
AST(SGOT)                    2390
Potassium                    2390
Bilirubin (Total)            2385
Chloride                     2227
Hematocrit                   2202
Hemoglobin                   2202
Glucose                      2199
Albumin                      2105
Protein                      2105
Calcium                      2104
Alkaline Phosphatase         2104
Phosphorus                   2104
Platelets                    2103
Bicarbonate                  2041
White Blood Cell (WBC)       2019
Red Blood Cells (RBC)        2017
CK                           1942
Urine Ph                     1865
Total Cholesterol            1864
Absolute Basophil Count      1864
Gamma-glutamyltransferase    1864
Basophils                    1803
Monocytes                    1803
Lymphocytes                  1803
Absolute Eosinophil Count    1745
E

In [100]:
all_feature_metadata = invert_func_to_features(ts_funcs_to_features, "ts")
all_feature_metadata.update(invert_func_to_features(dummy_funcs_to_features, "dummy"))
all_feature_metadata = learn_to_dummies_model(df, all_feature_metadata)
vectorized, all_feature_metadata = vectorize(df, all_feature_metadata, debug=True)


family_ALS_hist
Hematocrit
Prothrombin Time (clotting)
weight
Sodium
Absolute Band Neutrophil Count
hands
Total Cholesterol
pulse
White Blood Cell (WBC)
Monocytes
Free T4
Lymphocytes
Albumin
Chloride
Neutrophils
ALSFRS_Total
Glucose
Alkaline Phosphatase
bp_systolic
Bicarbonate
Potassium
Urine Specific Gravity
bp_diastolic
Absolute Eosinophil Count
Red Blood Cells (RBC)
Blood Urea Nitrogen (BUN)
Phosphorus
CK
Eosinophils
fvc_percent
Absolute Neutrophil Count
Gender
Age
respiratory_rate
HbA1c (Glycated Hemoglobin)
Absolute Monocyte Count
Calcium
Urine Ph
mouth
Gamma-glutamyltransferase
Lactate Dehydrogenase
Basophils
onset_delta
Bilirubin (Total)
Creatinine
BMI
temperature
respiratory
AST(SGOT)
Hemoglobin
Absolute Lymphocyte Count
Platelets
Triglycerides
Race
if_use_Riluzole
Absolute Basophil Count
Protein
Uric Acid
ALT(SGPT)


In [101]:
vectorized.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
family_ALS_hist_last,375,0.210667,0.408327,0.000000,0.000000,0.000000,0.000000,1.000000
Hematocrit_last,2212,39.442342,12.240731,0.359000,39.675000,42.600000,45.000000,69.000000
Hematocrit_mean_slope,1873,-0.007893,0.132361,-1.139583,-0.044444,-0.002694,0.034821,1.728571
Hematocrit_mean,2212,39.357237,12.147535,0.359000,39.645833,42.570833,45.000000,56.666667
Hematocrit_pct_diff,1873,0.000060,0.001476,-0.011247,-0.000458,0.000000,0.000503,0.040498
Prothrombin Time (clotting)_last,384,34.209635,36.901533,9.200000,10.600000,11.600000,84.250000,100.000000
Prothrombin Time (clotting)_mean_slope,76,-0.014852,0.130429,-0.984615,0.000000,0.000000,0.010417,0.133333
Prothrombin Time (clotting)_mean,384,34.222179,36.891253,9.200000,10.600000,11.666667,84.250000,100.000000
Prothrombin Time (clotting)_pct_diff,76,-0.000522,0.006362,-0.042078,0.000000,0.000000,0.000787,0.012987
weight_last,2424,75.956561,15.776150,37.700000,64.475000,75.000000,85.500000,147.500000


In [30]:
slope = pd.read_csv('../all_slope.csv', sep = '|', index_col=0)
slope.index = slope.index.astype(str)

max_date = df[df.feature_name == 'ALSFRS_Total'][['SubjectID','feature_delta']]
max_date.loc[:, 'feature_delta'] = max_date.feature_delta.astype(int)
max_date = max_date.groupby('SubjectID').max()
print max_date.shape, slope.shape
j = slope.join(max_date)
print j.shape
j[j.feature_delta < 365].shape

(2424, 1) (2424, 1)
(2424, 2)


(219, 2)

In [33]:
df[df.feature_name == 'onset_site'].feature_value.unique()

array(['Bulbar', 'Limb', 'Limb and Bulbar'], dtype=object)

In [7]:
slope = pd.read_csv('../all_slope.csv', sep = '|', index_col=0)
filtered = pd.read_csv('../../ALSFRS_slope_PROACT_filtered.txt', sep = '|', index_col=0)
filtered.head()

,ALSFRS_slope
SubjectID,
533,-0.965608
649,-0.921717
1234,-0.914787
2492,-0.598361
2956,-0.444039


In [8]:
slope.head()

,ALSFRS_slope
SubjectID,
533,-0.965608
649,-0.921717
1234,-0.914787
2492,-0.598361
2956,-0.444039


In [19]:
j = pd.merge(filtered, slope, how='right', left_index = True, right_index = True)

In [22]:
j[np.isnan(j.ALSFRS_slope_x)].head()

,ALSFRS_slope_x,ALSFRS_slope_y
SubjectID,,
11041,NaN,-0.404208
15154,NaN,-2.628601
85731,NaN,-0.824859
87664,NaN,-1.141923
112667,NaN,0.274024
